In [1]:
import numpy as np
import numpy.linalg as la
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import signal
import os
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [9]:
#save_path = 'D:\EECE499\Features\EEGFeatures.xlsx'
save_path = '..\..\..\EEGFeatures.xlsx'

In [4]:
sample_rate = 32

In [5]:
user_id = 1
clip_id = 1

data_path = './../ASCERTAIN_Raw/EEGData/Movie_P' + str(user_id).zfill(2) + '/EEG_Clip' + str(clip_id) + '.mat'

In [6]:
valid_user_clips = {
    '1' : list(range(1, 37)),
    '2' : list(range(1, 37)),
    '3' : list(range(1, 37)),
    '4' : list(range(1, 37)),
    '5' : list(range(1, 37)),
    '6' : list(range(1, 37)),
    '7' : list(range(1, 37)),
    '8' : list(range(1, 37)),
    '9' : list(range(1, 37)),
    '10' : list(range(1, 37)),
    '11' : list(range(1, 37)),
    '12' : list(range(1, 37)),
    '13' : list(range(1, 37)),
    '14' : list(range(1, 37)),
    '15' : list(range(1, 37)),
    '16' : list(range(1, 37)),
    '17' : list(range(1, 37)),
    '18' : list(range(1, 37)),
    '19' : list(range(1, 37)),
    '20' : list(range(1, 37)),
    '21' : list(range(1, 37)),
    '22' : list(range(1, 37)),
    '23' : list(range(1, 37)),
    '24' : list(range(1, 37)),
    '25' : list(range(1, 37)),
    '26' : list(range(1, 37)),
    '27' : list(range(1, 37)),
    '28' : list(range(1, 37)),
    '29' : list(range(1, 37)),
    '30' : list(range(1, 37)),
    '31' : list(range(1, 37)),
    '32' : list(range(1, 37)),
    '33' : list(range(1, 37)),
    '34' : list(range(1, 37)),
    '35' : list(range(1, 37)),
    '36' : list(range(1, 37)),
    '37' : list(range(1, 37)),
    '38' : list(range(1, 37)),
    '39' : list(range(1, 37)),
    '40' : list(range(1, 37)),
    '41' : list(range(1, 37)),
    '42' : list(range(1, 37)),
    '43' : list(range(1, 37)),
    '44' : list(range(1, 37)),
    '45' : list(range(1, 37)),
    '46' : list(range(1, 37)),
    '47' : list(range(1, 37)),
    '48' : list(range(1, 37)),
    '49' : list(range(1, 37)),
    '50' : list(range(1, 37)),
    '51' : list(range(1, 37)),
    '52' : list(range(1, 37)),
    '53' : list(range(1, 37)),
    '54' : list(range(1, 37)),
    '55' : list(range(1, 37)),
    '56' : list(range(1, 37)),
    '57' : list(range(1, 37)),
    '58' : list(range(1, 37))
}

#valid_user_clips['3'].remove(9)
#valid_user_clips['3'].remove(12)
#valid_user_clips['3'].remove(18)
#valid_user_clips['3'].remove(28)
#valid_user_clips['4'].remove(36)
#valid_user_clips['5'].remove(4)
#valid_user_clips['5'].remove(36)
#valid_user_clips['7'].remove(4)
#valid_user_clips['7'].remove(6)
#valid_user_clips['7'].remove(7)
#valid_user_clips['7'].remove(10)
#valid_user_clips['7'].remove(15)
#valid_user_clips['7'].remove(20)
#valid_user_clips['7'].remove(22)
#valid_user_clips['7'].remove(27)
#valid_user_clips['7'].remove(31)
#valid_user_clips['7'].remove(32)
#valid_user_clips['7'].remove(35)
#valid_user_clips['15'].remove(5)
#valid_user_clips['15'].remove(10)
#valid_user_clips['15'].remove(25)
#valid_user_clips['15'].remove(30)
#valid_user_clips['15'].remove(35)
#valid_user_clips['16'].remove(8)
#valid_user_clips['16'].remove(10)
#valid_user_clips['16'].remove(22)
#valid_user_clips['17'].remove(6)
#valid_user_clips['17'].remove(11)
#valid_user_clips['19'].remove(3)
#valid_user_clips['22'].remove(5)
#valid_user_clips['22'].remove(10)
#valid_user_clips['22'].remove(31)
#valid_user_clips['26'].remove(12)
#valid_user_clips['29'].remove(9)
#valid_user_clips['33'].remove(8)
#valid_user_clips['33'].remove(22)
#valid_user_clips['34'].remove(8)
#valid_user_clips['34'].remove(21)
#valid_user_clips['37'].remove(16)
#valid_user_clips['37'].remove(19)
#valid_user_clips['37'].remove(23)
#valid_user_clips['37'].remove(26)
#valid_user_clips['39'].remove(28)
#valid_user_clips['40'].remove(3)
#valid_user_clips['41'].remove(7)
#valid_user_clips['43'].remove(4)
#valid_user_clips['43'].remove(13)
#valid_user_clips['43'].remove(14)
#valid_user_clips['45'].remove(7)
#valid_user_clips['45'].remove(15)

In [8]:
data_frames = []

for user_id in range(1, 59):
    print('User ID: ' + str(user_id))

    neuro_sky_EEG = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    attention = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    meditation = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    alpha = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    beta = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    delta = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    gamma = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    theta = {
        'avg_first_deriv' : [],
        'percent_neg_deriv' : [],
        'mean_peaks' : [],
        'avg_first_deriv_inv' : [],
        'mean_peaks_inv' : [],
        'mean' : [],
        'std' : [],
        'skewness' : [],
        'kurtosis' : [],
        'percent_over_std' : [],
        'percent_under_std' : []
    }
    
    presentation_id = []
    
    for clip_id in valid_user_clips[str(user_id)]:
        #print('Clip ID: ' + str(clip_id))
        
        data_path = './../../ASCERTAIN_Raw/EEGData/Movie_P' + str(user_id).zfill(2) + '/EEG_Clip' + str(clip_id) + '.mat'
        
        presentation_id.append(str(user_id).zfill(2) + str(clip_id).zfill(2))
        
        signal = scipy.io.loadmat(data_path)
        neuro_sky_EEG_raw = np.array(signal['ThisEEG'][0])
        attention_raw = np.array(signal['ThisEEG'][1])
        meditation_raw = np.array(signal['ThisEEG'][2])
        alpha_raw = np.array(signal['ThisEEG'][3])
        beta_raw = np.array(signal['ThisEEG'][4])
        delta_raw = np.array(signal['ThisEEG'][5])
        gamma_raw = np.array(signal['ThisEEG'][6])
        theta_raw = np.array(signal['ThisEEG'][7])
        
        #neuro_sky_EEG_raw /= la.norm(neuro_sky_EEG_raw)
        #attention_raw /= la.norm(attention_raw)
        #meditation_raw /= la.norm(meditation_raw)
        #alpha_raw /= la.norm(alpha_raw)
        #beta_raw /= la.norm(beta_raw)
        #delta_raw /= la.norm(delta_raw)
        #gamma_raw /= la.norm(gamma_raw)
        #theta_raw /= la.norm(theta_raw)
        #
        #neuro_sky_EEG_raw = np.abs(neuro_sky_EEG_raw)
        #attention_raw = np.abs(attention_raw)
        #meditation_raw = np.abs(meditation_raw)
        #alpha_raw = np.abs(alpha_raw)
        #beta_raw = np.abs(beta_raw)
        #delta_raw = np.abs(delta_raw)
        #gamma_raw = np.abs(gamma_raw)
        #theta_raw = np.abs(theta_raw)
        
        #plt.plot(neuro_sky_EEG_raw, color = 'blue')
        window = 51 #101
        order = 4 #3
        try:
            neuro_sky_EEG_raw = savgol_filter(neuro_sky_EEG_raw, window, order)
            attention_raw = savgol_filter(attention_raw, window, order)
            meditation_raw = savgol_filter(meditation_raw, window, order)
            alpha_raw = savgol_filter(alpha_raw, window, order)
            beta_raw = savgol_filter(beta_raw, window, order)
            delta_raw = savgol_filter(delta_raw, window, order)
            gamma_raw = savgol_filter(gamma_raw, window, order)
            theta_raw = savgol_filter(theta_raw, window, order)
        except:
            l = len(neuro_sky_EEG_raw)
            neuro_sky_EEG_raw = np.zeros(l)
            attention_raw = np.zeros(l)
            meditation_raw = np.zeros(l)
            alpha_raw = np.zeros(l)
            beta_raw = np.zeros(l)
            delta_raw = np.zeros(l)
            gamma_raw = np.zeros(l)
            theta_raw = np.zeros(l)
        #plt.plot(neuro_sky_EEG_raw, color = 'red')
        #plt.show()

        if os.path.isfile(data_path):
            
            #plt.plot(attention_raw)
            #plt.show()
            #print(neuro_sky_EEG_raw)
            
            # average of the first derivative
            neuro_sky_EEG['avg_first_deriv'].append(np.mean(np.diff(neuro_sky_EEG_raw) / np.diff(range(neuro_sky_EEG_raw.size))))
            attention['avg_first_deriv'].append(np.mean(np.diff(attention_raw) / np.diff(range(attention_raw.size))))
            meditation['avg_first_deriv'].append(np.mean(np.diff(meditation_raw) / np.diff(range(meditation_raw.size))))
            alpha['avg_first_deriv'].append(np.mean(np.diff(alpha_raw) / np.diff(range(alpha_raw.size))))
            beta['avg_first_deriv'].append(np.mean(np.diff(beta_raw) / np.diff(range(beta_raw.size))))
            delta['avg_first_deriv'].append(np.mean(np.diff(delta_raw) / np.diff(range(delta_raw.size))))
            gamma['avg_first_deriv'].append(np.mean(np.diff(gamma_raw) / np.diff(range(gamma_raw.size))))
            theta['avg_first_deriv'].append(np.mean(np.diff(theta_raw) / np.diff(range(theta_raw.size))))
            #print(neuro_sky_EEG)
            
            # percentage of negative differential samples
            deriv = np.diff(neuro_sky_EEG_raw) / np.diff(range(neuro_sky_EEG_raw.size))
            neuro_sky_EEG['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(attention_raw) / np.diff(range(attention_raw.size))
            attention['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(meditation_raw) / np.diff(range(meditation_raw.size))
            meditation['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(alpha_raw) / np.diff(range(alpha_raw.size))
            alpha['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(beta_raw) / np.diff(range(beta_raw.size))
            beta['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(delta_raw) / np.diff(range(delta_raw.size))
            delta['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(gamma_raw) / np.diff(range(gamma_raw.size))
            gamma['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            deriv = np.diff(theta_raw) / np.diff(range(theta_raw.size))
            theta['percent_neg_deriv'].append(len(deriv[deriv < 0]) / len(deriv))
            
            # mean number of peaks
            peaks, _ = find_peaks(neuro_sky_EEG_raw)
            neuro_sky_EEG['mean_peaks'].append(len(peaks) / len(neuro_sky_EEG_raw))
            peaks, _ = find_peaks(attention_raw)
            attention['mean_peaks'].append(len(peaks) / len(attention_raw))
            peaks, _ = find_peaks(meditation_raw)
            meditation['mean_peaks'].append(len(peaks) / len(meditation_raw))
            peaks, _ = find_peaks(alpha_raw)
            alpha['mean_peaks'].append(len(peaks) / len(alpha_raw))
            peaks, _ = find_peaks(beta_raw)
            beta['mean_peaks'].append(len(peaks) / len(beta_raw))
            peaks, _ = find_peaks(delta_raw)
            delta['mean_peaks'].append(len(peaks) / len(delta_raw))
            peaks, _ = find_peaks(gamma_raw)
            gamma['mean_peaks'].append(len(peaks) / len(gamma_raw))
            peaks, _ = find_peaks(theta_raw)
            theta['mean_peaks'].append(len(peaks) / len(theta_raw))
            #print(neuro_sky_EEG['mean_peaks'])
            
            # average of first derivative of inverse signal
            neuro_sky_EEG['avg_first_deriv_inv'].append(np.mean(np.diff(1 / neuro_sky_EEG_raw) / np.diff(range(neuro_sky_EEG_raw.size))))
            attention['avg_first_deriv_inv'].append(np.mean(np.diff(1 / attention_raw) / np.diff(range(attention_raw.size))))
            meditation['avg_first_deriv_inv'].append(np.mean(np.diff(1 / meditation_raw) / np.diff(range(meditation_raw.size))))
            alpha['avg_first_deriv_inv'].append(np.mean(np.diff(1 / alpha_raw) / np.diff(range(alpha_raw.size))))
            beta['avg_first_deriv_inv'].append(np.mean(np.diff(1 / beta_raw) / np.diff(range(beta_raw.size))))
            delta['avg_first_deriv_inv'].append(np.mean(np.diff(1 / delta_raw) / np.diff(range(delta_raw.size))))
            gamma['avg_first_deriv_inv'].append(np.mean(np.diff(1 / gamma_raw) / np.diff(range(gamma_raw.size))))
            theta['avg_first_deriv_inv'].append(np.mean(np.diff(1 / theta_raw) / np.diff(range(theta_raw.size))))
            
            # mean number of peaks in inverse signal
            peaks, _ = find_peaks(1 / neuro_sky_EEG_raw)
            neuro_sky_EEG['mean_peaks_inv'].append(len(peaks) / len(neuro_sky_EEG_raw))
            peaks, _ = find_peaks(1 / attention_raw)
            attention['mean_peaks_inv'].append(len(peaks) / len(attention_raw))
            peaks, _ = find_peaks(1 / meditation_raw)
            meditation['mean_peaks_inv'].append(len(peaks) / len(meditation_raw))
            peaks, _ = find_peaks(1 / alpha_raw)
            alpha['mean_peaks_inv'].append(len(peaks) / len(alpha_raw))
            peaks, _ = find_peaks(1 / beta_raw)
            beta['mean_peaks_inv'].append(len(peaks) / len(beta_raw))
            peaks, _ = find_peaks(1 / delta_raw)
            delta['mean_peaks_inv'].append(len(peaks) / len(delta_raw))
            peaks, _ = find_peaks(1 / gamma_raw)
            gamma['mean_peaks_inv'].append(len(peaks) / len(gamma_raw))
            peaks, _ = find_peaks(1 / theta_raw)
            theta['mean_peaks_inv'].append(len(peaks) / len(theta_raw))
            
            # mean of signal
            neuro_sky_EEG['mean'].append(np.mean(neuro_sky_EEG_raw))
            attention['mean'].append(np.mean(attention_raw))
            meditation['mean'].append(np.mean(meditation_raw))
            alpha['mean'].append(np.mean(alpha_raw))
            beta['mean'].append(np.mean(beta_raw))
            delta['mean'].append(np.mean(delta_raw))
            gamma['mean'].append(np.mean(gamma_raw))
            theta['mean'].append(np.mean(theta_raw))
            
            # standard deviation of signal
            neuro_sky_EEG['std'].append(np.std(neuro_sky_EEG_raw))
            attention['std'].append(np.std(attention_raw))
            meditation['std'].append(np.std(meditation_raw))
            alpha['std'].append(np.std(alpha_raw))
            beta['std'].append(np.std(beta_raw))
            delta['std'].append(np.std(delta_raw))
            gamma['std'].append(np.std(gamma_raw))
            theta['std'].append(np.std(theta_raw))
            
            # skewness of signal
            neuro_sky_EEG['skewness'].append(skew(neuro_sky_EEG_raw))
            attention['skewness'].append(skew(attention_raw))
            meditation['skewness'].append(skew(meditation_raw))
            alpha['skewness'].append(skew(alpha_raw))
            beta['skewness'].append(skew(beta_raw))
            delta['skewness'].append(skew(delta_raw))
            gamma['skewness'].append(skew(gamma_raw))
            theta['skewness'].append(skew(theta_raw))
            
            # kurtosis of signal
            neuro_sky_EEG['kurtosis'].append(kurtosis(neuro_sky_EEG_raw))
            attention['kurtosis'].append(kurtosis(attention_raw))
            meditation['kurtosis'].append(kurtosis(meditation_raw))
            alpha['kurtosis'].append(kurtosis(alpha_raw))
            beta['kurtosis'].append(kurtosis(beta_raw))
            delta['kurtosis'].append(kurtosis(delta_raw))
            gamma['kurtosis'].append(kurtosis(gamma_raw))
            theta['kurtosis'].append(kurtosis(theta_raw))
            
            # percent of samples above the mean + std
            neuro_sky_EEG['percent_over_std'].append(len(neuro_sky_EEG_raw[neuro_sky_EEG_raw > np.mean(neuro_sky_EEG_raw) + np.std(neuro_sky_EEG_raw)]) / len(neuro_sky_EEG_raw))
            attention['percent_over_std'].append(len(attention_raw[attention_raw > np.mean(attention_raw) + np.std(attention_raw)]) / len(attention_raw))
            meditation['percent_over_std'].append(len(meditation_raw[meditation_raw > np.mean(meditation_raw) + np.std(meditation_raw)]) / len(meditation_raw))
            alpha['percent_over_std'].append(len(alpha_raw[alpha_raw > np.mean(alpha_raw) + np.std(alpha_raw)]) / len(alpha_raw))
            beta['percent_over_std'].append(len(beta_raw[beta_raw > np.mean(beta_raw) + np.std(beta_raw)]) / len(beta_raw))
            delta['percent_over_std'].append(len(delta_raw[delta_raw > np.mean(delta_raw) + np.std(delta_raw)]) / len(delta_raw))
            gamma['percent_over_std'].append(len(gamma_raw[gamma_raw > np.mean(gamma_raw) + np.std(gamma_raw)]) / len(gamma_raw))
            theta['percent_over_std'].append(len(theta_raw[theta_raw > np.mean(theta_raw) + np.std(theta_raw)]) / len(theta_raw))
            
            # percent of samples below mean - std
            neuro_sky_EEG['percent_under_std'].append(len(neuro_sky_EEG_raw[neuro_sky_EEG_raw < np.mean(neuro_sky_EEG_raw) - np.std(neuro_sky_EEG_raw)]) / len(neuro_sky_EEG_raw))
            attention['percent_under_std'].append(len(attention_raw[attention_raw < np.mean(attention_raw) - np.std(attention_raw)]) / len(attention_raw))
            meditation['percent_under_std'].append(len(meditation_raw[meditation_raw < np.mean(meditation_raw) - np.std(meditation_raw)]) / len(meditation_raw))
            alpha['percent_under_std'].append(len(alpha_raw[alpha_raw < np.mean(alpha_raw) - np.std(alpha_raw)]) / len(alpha_raw))
            beta['percent_under_std'].append(len(beta_raw[beta_raw < np.mean(beta_raw) - np.std(beta_raw)]) / len(beta_raw))
            delta['percent_under_std'].append(len(delta_raw[delta_raw < np.mean(delta_raw) - np.std(delta_raw)]) / len(delta_raw))
            gamma['percent_under_std'].append(len(gamma_raw[gamma_raw < np.mean(gamma_raw) - np.std(gamma_raw)]) / len(gamma_raw))
            theta['percent_under_std'].append(len(theta_raw[theta_raw < np.mean(theta_raw) - np.std(theta_raw)]) / len(theta_raw))
        
    #for key, value in neuro_sky_EEG.items():
    #    print('neuro_sky_EEG.' + key + '=' + str(len(value)))
    #for key, value in attention.items():
    #    print('attention.' + key + '=' + str(len(value)))
    #for key, value in meditation.items():
    #    print('meditation.' + key + '=' + str(len(value)))
    #for key, value in alpha.items():
    #    print('alpha.' + key + '=' + str(len(value)))
    #for key, value in beta.items():
    #    print('beta.' + key + '=' + str(len(value)))
    #for key, value in delta.items():
    #    print('delta.' + key + '=' + str(len(value)))
    #for key, value in gamma.items():
    #    print('gamma.' + key + '=' + str(len(value)))
    #for key, value in theta.items():
    #    print('theta.' + key + '=' + str(len(value)))

    # saving all the data into dataframe
    user_data = pd.DataFrame({
        'presentation_id' : presentation_id,
        'neur_sky_avg_first_deriv' : neuro_sky_EEG['avg_first_deriv'],
        'neur_sky_percent_neg_deriv' : neuro_sky_EEG['percent_neg_deriv'],
        'neur_sky_mean_peaks' : neuro_sky_EEG['mean_peaks'],
        'neur_sky_avg_first_deriv_inv' : neuro_sky_EEG['avg_first_deriv_inv'],
        'neur_sky_mean_peaks_inv' : neuro_sky_EEG['mean_peaks_inv'],
        'neur_sky_mean' : neuro_sky_EEG['mean'],
        'neur_sky_std' : neuro_sky_EEG['std'],
        'neur_sky_skewness' : neuro_sky_EEG['skewness'],
        'neur_sky_kurtosis' : neuro_sky_EEG['kurtosis'],
        'neur_sky_percent_over_std' : neuro_sky_EEG['percent_over_std'],
        'neur_sky_percent_under_std' : neuro_sky_EEG['percent_under_std'],
        'attention_avg_first_deriv' : attention['avg_first_deriv'],
        'attention_percent_neg_deriv' : attention['percent_neg_deriv'],
        'attention_mean_peaks' : attention['mean_peaks'],
        'attention_avg_first_deriv_inv' : attention['avg_first_deriv_inv'],
        'attention_mean_peaks_inv' : attention['mean_peaks_inv'],
        'attention_mean' : attention['mean'],
        'attention_std' : attention['std'],
        'attention_skewness' : attention['skewness'],
        'attention_kurtosis' : attention['kurtosis'],
        'attention_percent_over_std' : attention['percent_over_std'],
        'attention_percent_under_std' : attention['percent_under_std'],
        'meditation_avg_first_deriv' : meditation['avg_first_deriv'],
        'meditation_percent_neg_deriv' : meditation['percent_neg_deriv'],
        'meditation_mean_peaks' : meditation['mean_peaks'],
        'meditation_avg_first_deriv_inv' : meditation['avg_first_deriv_inv'],
        'meditation_mean_peaks_inv' : meditation['mean_peaks_inv'],
        'meditation_mean' : meditation['mean'],
        'meditation_std' : meditation['std'],
        'meditation_skewness' : meditation['skewness'],
        'meditation_kurtosis' : meditation['kurtosis'],
        'meditation_percent_over_std' : meditation['percent_over_std'],
        'meditation_percent_under_std' : meditation['percent_under_std'],
        'alpha_avg_first_deriv' : alpha['avg_first_deriv'],
        'alpha_percent_neg_deriv' : alpha['percent_neg_deriv'],
        'alpha_mean_peaks' : alpha['mean_peaks'],
        'alpha_avg_first_deriv_inv' : alpha['avg_first_deriv_inv'],
        'alpha_mean_peaks_inv' : alpha['mean_peaks_inv'],
        'alpha_mean' : alpha['mean'],
        'alpha_std' : alpha['std'],
        'alpha_skewness' : alpha['skewness'],
        'alpha_kurtosis' : alpha['kurtosis'],
        'alpha_percent_over_std' : alpha['percent_over_std'],
        'alpha_percent_under_std' : alpha['percent_under_std'],
        'beta_avg_first_deriv' : beta['avg_first_deriv'],
        'beta_percent_neg_deriv' : beta['percent_neg_deriv'],
        'beta_mean_peaks' : beta['mean_peaks'],
        'beta_avg_first_deriv_inv' : beta['avg_first_deriv_inv'],
        'beta_mean_peaks_inv' : beta['mean_peaks_inv'],
        'beta_mean' : beta['mean'],
        'beta_std' : beta['std'],
        'beta_skewness' : beta['skewness'],
        'beta_kurtosis' : beta['kurtosis'],
        'beta_percent_over_std' : beta['percent_over_std'],
        'beta_percent_under_std' : beta['percent_under_std'],
        'delta_avg_first_deriv' : delta['avg_first_deriv'],
        'delta_percent_neg_deriv' : delta['percent_neg_deriv'],
        'delta_mean_peaks' : delta['mean_peaks'],
        'delta_avg_first_deriv_inv' : delta['avg_first_deriv_inv'],
        'delta_mean_peaks_inv' : delta['mean_peaks_inv'],
        'delta_mean' : delta['mean'],
        'delta_std' : delta['std'],
        'delta_skewness' : delta['skewness'],
        'delta_kurtosis' : delta['kurtosis'],
        'delta_percent_over_std' : delta['percent_over_std'],
        'delta_percent_under_std' : delta['percent_under_std'],
        'gamma_avg_first_deriv' : gamma['avg_first_deriv'],
        'gamma_percent_neg_deriv' : gamma['percent_neg_deriv'],
        'gamma_mean_peaks' : gamma['mean_peaks'],
        'gamma_avg_first_deriv_inv' : gamma['avg_first_deriv_inv'],
        'gamma_mean_peaks_inv' : gamma['mean_peaks_inv'],
        'gamma_mean' : gamma['mean'],
        'gamma_std' : gamma['std'],
        'gamma_skewness' : gamma['skewness'],
        'gamma_kurtosis' : gamma['kurtosis'],
        'gamma_percent_over_std' : gamma['percent_over_std'],
        'gamma_percent_under_std' : gamma['percent_under_std'],
        'theta_avg_first_deriv' : theta['avg_first_deriv'],
        'theta_percent_neg_deriv' : theta['percent_neg_deriv'],
        'theta_mean_peaks' : theta['mean_peaks'],
        'theta_avg_first_deriv_inv' : theta['avg_first_deriv_inv'],
        'theta_mean_peaks_inv' : theta['mean_peaks_inv'],
        'theta_mean' : theta['mean'],
        'theta_std' : theta['std'],
        'theta_skewness' : theta['skewness'],
        'theta_kurtosis' : theta['kurtosis'],
        'theta_percent_over_std' : theta['percent_over_std']#,
        #'theta_percent_under_std' : theta['percent_under_std']
    })
        
    data_frames.append(user_data)

User ID: 1


C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:198: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:202: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:204: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:299: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:300: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:301: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:302: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib

User ID: 2


C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:228: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:229: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:230: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:231: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:232: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:234: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:235: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:238: RuntimeWarning: divide by zero enc

User ID: 3


...

User ID: 4
User ID: 5
User ID: 6
User ID: 7


C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:206: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:303: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:313: RuntimeWarning: invalid value encountered in less


User ID: 8


C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:196: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:297: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:298: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:307: RuntimeWarning: invalid value encountered in less
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:308: RuntimeWarning: invalid value encountered in less


User ID: 9
User ID: 10
User ID: 11
User ID: 12
User ID: 13
User ID: 14
User ID: 15
User ID: 16
User ID: 17
User ID: 18
User ID: 19
User ID: 20
User ID: 21
User ID: 22
User ID: 23
User ID: 24
User ID: 25
User ID: 26
User ID: 27
User ID: 28
User ID: 29
User ID: 30
User ID: 31
User ID: 32
User ID: 33
User ID: 34
User ID: 35
User ID: 36
User ID: 37
User ID: 38
User ID: 39
User ID: 40
User ID: 41
User ID: 42
User ID: 43
User ID: 44
User ID: 45
User ID: 46
User ID: 47
User ID: 48
User ID: 49
User ID: 50
User ID: 51
User ID: 52
User ID: 53
User ID: 54
User ID: 55
User ID: 56
User ID: 57
User ID: 58


In [10]:
EEG_Features = pd.concat(data_frames,ignore_index=True)

In [11]:
EEG_Features.to_excel(save_path)

data_path = './../ASCERTAIN_Features/ASCERTAIN_Features/Dt_EEGFeatures.mat'
signal = scipy.io.loadmat(data_path)
print(signal['EEGFeatures_58'][0, 0][:, 11])
print(attention['mean'])

mean = np.array(signal['EEGFeatures_58'][0, 0][:, 0])
plt.plot(neuro_sky_EEG['mean'])
plt.plot(mean, color='red')
plt.show()
std = np.array(signal['EEGFeatures_58'][0, 0][:, 1])
plt.plot(neuro_sky_EEG['std'])
plt.plot(std, color='red')
plt.show()
skew = np.array(signal['EEGFeatures_58'][0, 0][:, 2])
plt.plot(neuro_sky_EEG['skewness'])
plt.plot(skew, color='red')
plt.show()
kurtosis = np.array(signal['EEGFeatures_58'][0, 0][:, 3])
plt.plot(neuro_sky_EEG['kurtosis'])
plt.plot(kurtosis, color='red')
plt.show()

In [12]:
EEG_Features

,alpha_avg_first_deriv,alpha_avg_first_deriv_inv,alpha_kurtosis,alpha_mean,alpha_mean_peaks,alpha_mean_peaks_inv,alpha_percent_neg_deriv,alpha_percent_over_std,alpha_percent_under_std,alpha_skewness,...,theta_avg_first_deriv,theta_avg_first_deriv_inv,theta_kurtosis,theta_mean,theta_mean_peaks,theta_mean_peaks_inv,theta_percent_neg_deriv,theta_percent_over_std,theta_skewness,theta_std
0,-7.190065e-03,6.831202e-08,0.318642,224.736879,0.011019,0.011019,0.604103,0.178145,0.178757,0.944644,...,-7.884726e-03,2.089398e-07,0.460582,154.853318,0.012856,0.012550,0.753521,0.137435,0.695442,5.821595e+01
1,3.043641e-02,-1.981346e-06,-1.232518,319.581927,0.005592,0.005592,0.772494,0.240447,0.207829,0.025465,...,-2.829065e-01,1.359779e-05,4.551899,104.597244,0.020037,0.019571,0.810723,0.123486,2.235774,1.276209e+02
2,-1.153553e-02,5.937411e-07,-0.568483,152.523052,0.007946,0.008513,0.734810,0.195800,0.144722,0.595697,...,-2.461746e-02,1.160076e-05,0.129418,41.658683,0.006810,0.007378,0.778535,0.163451,0.858342,1.674968e+01
3,-1.466625e-01,7.710328e-06,5.418930,97.556580,0.020558,0.020925,0.343371,0.118943,0.000000,2.520702,...,-4.276875e-02,1.567088e-05,-1.395188,82.669052,0.014317,0.014317,0.709512,0.248532,-0.210633,4.331268e+01
4,-1.948462e-01,5.199676e-07,-1.523223,707.213232,0.007493,0.007493,0.431293,0.000000,0.274118,-0.477477,...,-1.452940e-01,7.964896e-07,-1.479756,550.077516,0.009054,0.009054,0.719238,0.206681,-0.029034,3.164298e+02
5,-1.615151e-02,2.432054e-07,17.921853,270.425010,0.001200,0.001200,0.090691,0.000000,0.063625,-4.282814,...,-9.391675e-03,3.445066e-08,20.306134,529.457110,0.001200,0.001200,0.088889,0.000000,-4.522001,2.364671e+00
6,-8.503619e-02,8.898201e-07,1.760498,126.241152,0.004509,0.004509,0.346287,0.113770,0.000000,1.321820,...,2.112298e-04,-2.259633e-08,-1.436177,216.701065,0.003469,0.003122,0.673144,0.278529,-0.228088,1.203554e+02
7,-2.650043e-17,9.704747e-21,37.954016,50.500000,0.000466,0.000932,0.004662,0.023299,0.000000,6.321321,...,-1.592003e-02,3.205592e-05,8.301979,13.388698,0.001398,0.001398,0.145921,0.100652,3.052270,7.017808e+00
8,9.076462e-03,-1.376053e-07,-0.169159,173.298425,0.008467,0.008467,0.743201,0.191622,0.129234,0.791496,...,-1.074342e-01,2.696676e-06,-0.774325,156.268805,0.005348,0.005348,0.868034,0.220143,0.575253,9.509273e+01
9,NaN,NaN,NaN,NaN,0.000347,0.000347,0.057946,0.000000,0.000000,NaN,...,-6.903260e-17,3.310541e-21,52.690660,143.500000,0.000694,0.000347,0.003123,0.017343,7.395753,7.383958e-11
